In [ ]:
#samples directory 만들기

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist= input_data.read_data_sets('./mnist/data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [2]:
total_epoch = 100
batch_size=100
learning_rate = 0.0002
n_hidden = 256
n_input= 28*28
n_noise = 128

X= tf.placeholder(tf.float32, [None,n_input])  #>> 실제데이터 입력
Z = tf.placeholder(tf.float32, [None,n_noise]) #> noise 입력 

In [3]:
# 128x256
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden],stddev=0.01))

G_b1 = tf.Variable(tf.zeros([n_hidden]))
#256x784 
G_W2 = tf.Variable(tf.random_normal([n_hidden,n_input], stddev=0.01))

G_b2 = tf.Variable(tf.zeros([n_input]))
D_W1 = tf.Variable(tf.random_normal([n_input,n_hidden], stddev=0.01))

D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [4]:
# 생성기 외 판별기
# cost function : 확률적 함수
# 지금까지 MSE, Enthropy(이건분류일때) 를 사용했음  
# KL-Divergence  : GAN, VAE(분포의 차를 확인하는 함수이다 ) 에서는 확률적 함수를 사용한다 .
# 생성기 -> noise 가 input 데이터이다,  cost function 으로 KL-divergence 를 사용한다 .  ( 이것들이 GAN 의 특징이다 ) 
# ( 생성기입력에 noise 가 들어간다는것과 cost function 으로 Kl_divergence 를 사용한다 ) 

def generator(noise_z):  # 128x128
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1) # 128x256
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2) # 128x784 
    return output# 784 이미지가 생성된다 

def discriminator(inputs): # 여기서는 실제이미지의 분포를 확인하낟 
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1)+D_b1)  # input 은 784 
    output = tf.nn.sigmoid(tf.matmul(hidden,D_W2)+D_b2)
    return output

def get_noise(batch_size,n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [5]:

G = generator(Z)  # 제너레이터에서 노이즈를 가지고 이미지를 생성한다 
D_gene = discriminator(G)# >> 노이즈로부터 생성된 이미지를 판별한다 = 분포를 확인한다 
D_real = discriminator(X) # 실제이미지 X 의 분포를 확인한다

# 아래 로그가 붙는이유는? log 로 확률값을 구하는데 정보량때문이다
# 확률이 높아지면 정보량이 작아진다. / 확률이 낮아지면 정보량이 커진다 ( tfidf 의 논리를 고려할 것 ) 
# 가장 적합한 분포를 찾아내야한다 ( 높은 것이 유리하다 )  
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene))# 최대우도 추정법을 사용해서 적합한 분포가 될수있도록 로그를 취해서 평균을 내고 minimize 를 해주게 된다 
D_var_list = [D_W1, D_b1, D_W2, D_b2]     # ( 최대우도 추정법이니까 확률을 높일수록 좋다 ) 
G_var_list = [G_W1, G_b1, G_W2, G_b2]
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list= D_var_list) # 최대값을 찾아야하므로 미니마이즈쓰고 마이너스를 써줬다 
# 분포에 적합한, 높은것을 찾아내야하므로 
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,var_list=G_var_list) # list 를 쓰는건 minimize 를 할때 역전파 변수를 지정한것이다 
%matplotlib inline
sess = tf.Session() 
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0,0

In [6]:
# 이미지 분포의 특징을 추출하는 망이다!!! 
for epoch in range(total_epoch):                    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        # 실제 이미지 분포 
        _, loss_val_D = sess.run([train_D, loss_D],
                                feed_dict = {X: batch_xs, Z: noise})  # noise 가 들어가서 이미지를 생성한다   
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise}) # 배치사이즈 128 이니까 이미지가 128x128 로 생성된다 (( 위에 Z placeholder) 
    print('Epoch:',"%04d" % epoch,
         'D loss : {:.4}'.format(loss_val_D),
         'G loss : {:.4}'.format(loss_val_G))
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)  # 10x128 이다  >> noise 개수가 128 이다. ( n_noise 가 128, 배치사이즈 128) 
        samples = sess.run(G, feed_dict={Z:noise}) # Z 에 noise 를 주었다 
        fig,ax = plt.subplots(1, sample_size, figsize=(sample_size,1))
        for i in range(sample_size):
            ax[i].set_axis_off()  # noise 128개가 들어가게되면 784 이미지 사이즈로 생성이 된다. 
            ax[i].imshow(np.reshape(samples[i], (28,28)))  #
        plt.savefig('sampels/{}.png'.format(str(epoch).zfill(3)),
                       bbpx_inches ='tight')
        plt.close(fig)
print('최적화 완료')


Epoch: 0000 D loss : -0.645 G loss : -2.044
Epoch: 0001 D loss : -0.3066 G loss : -2.46
Epoch: 0002 D loss : -0.1659 G loss : -2.974
Epoch: 0003 D loss : -0.652 G loss : -1.444
Epoch: 0004 D loss : -0.3426 G loss : -1.881
Epoch: 0005 D loss : -0.2688 G loss : -2.413
Epoch: 0006 D loss : -0.3233 G loss : -2.729
Epoch: 0007 D loss : -0.2922 G loss : -2.455
Epoch: 0008 D loss : -0.2157 G loss : -2.403
Epoch: 0009 D loss : -0.2211 G loss : -2.614
Epoch: 0010 D loss : -0.2948 G loss : -2.504
Epoch: 0011 D loss : -0.3405 G loss : -2.526
Epoch: 0012 D loss : -0.4143 G loss : -2.15
Epoch: 0013 D loss : -0.4465 G loss : -2.16
Epoch: 0014 D loss : -0.4221 G loss : -2.235
Epoch: 0015 D loss : -0.2481 G loss : -2.693
Epoch: 0016 D loss : -0.3741 G loss : -2.666
Epoch: 0017 D loss : -0.3256 G loss : -2.506
Epoch: 0018 D loss : -0.3325 G loss : -2.628
Epoch: 0019 D loss : -0.3708 G loss : -2.64
Epoch: 0020 D loss : -0.289 G loss : -3.015
Epoch: 0021 D loss : -0.3997 G loss : -2.881
Epoch: 0022 D los

In [ ]:
# GAN 
## smaples폴더 에 있는 이미지를 보면 처음에는 뭉개져있지만  점점 뚜렷하게 예측하는 것을 볼 수 있다
## 중요한것은 noise 를 집어넣었다는 것이다 ( noise 가 어떤 수인지는 모른다 ) 
## noise 로 부터 이미지를 생성했다 (가중치를 곱해서 나온 generator 가 noise 로 부터 생성했더니 이미지가 나옴 )  # 괄호안이 맞는진모르겠음
## vae 보다 더 다양하게 사용가능
## 다양하게 활용이 되고 있다 